## 1. Linear regression vs ANN
   #  Manual feature engineering
   #  How ANN engineers features from inputs?
   

## Neural Network Basics
# 1. Introduction to Keras
 # 2.Keras Sequential NN model

# Compare the effectiveness ANN ultiple Regresiom.

# Demonnstrate that a better understanding of the problem to be solved allows engineering features that can enriches the inputs information and enhance the performance of ANN


## Keras created by Google as a high-level neural network API. 
 # It simplies deep learning and artificial intelligence application. 
  # It runs on top of a number of lower-level libraries, used as backends, including TensorFlow.

## 1. Linear regression vs ANN

# Import some libraries

In [1]:
## First we need to update numpy

In [4]:
pip install numpy==1.23.4

  Using cached numpy-1.23.4-cp38-cp38-macosx_10_9_x86_64.whl (18.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.23.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.4 which is incompatible.
bokeh 2.4.3 requires typing-extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
import statsmodels.api as sm
import statsmodels.formula.api as smf

ImportError: this version of pandas is incompatible with numpy < 1.20.3
your numpy version is 1.19.5.
Please upgrade numpy to >= 1.20.3 to use this pandas version

# Let us list files in our working directory

In [ ]:
!ls

In [ ]:
d0 =  pd.read_csv('top10_ret_volume_2023-05-24.csv')
d1  =  pd.read_csv('top10_ret_volume_2023-06-17.csv') 

In [ ]:
d0.columns

In [ ]:
d0_ = d0 [ d0['day'] < '2023-05-23']
d1_ = d1 [ d1['day'] > '2023-05-22']

In [ ]:
data0 = pd.concat([d0_,d1_])

In [ ]:
data0.shape

In [ ]:
data0[['symbol','etf' , 'day' , 'time' ]].describe()
    

In [ ]:
data0['symbol'].value_counts()

In [ ]:
data0['etf'].value_counts()

In [ ]:
data0['time'].value_counts()

## we have 5-minutes return and volume data for 53, for each symbol, daysstarting at 9:30 and ending at 15:55
# Let us filer out time = 16:00:00

In [ ]:
data = data0[data0['time']< '16:00:00']
print ( data0.shape, data.shape)

In [ ]:
data['symbol'].unique()

In [ ]:
by_sym_day_volume = data.groupby(['symbol','day'])['volume'].sum().reset_index()

In [ ]:
by_sym_volume = by_sym_day_volume.groupby('symbol').median().reset_index()
by_sym_volume = by_sym_volume.sort_values('volume', ascending=False)

In [ ]:
syms_ = by_sym_volume.head(30)


In [ ]:
syms=list( syms_['symbol'] )
print(syms)

## In our analysis we will focus the above 30 stock

In [ ]:
syms0= data[data['symbol'].isin(syms)]
print( data0.shape, syms0.shape )

In [ ]:
syms0.columns

In [ ]:
sym0.isnull().sum()

## let us focus on a subset of the columns

In [ ]:
syms0.columns

## Note that,
 # ret          is the stock's return over the 5-minutes ending at time t
 # volume  is the stock's volume over the 5-minutes ending at time t
## next_ret   is the stock's return over the 5-minutes starting at time t
 # At time t,  ret and volume are know, but next_ret is not know.

## Suppose that we want to predict next_ret at time t using information at t ( i.e. [ret, volume])

## let us look at the following correlation matrix

In [ ]:
syms0[['next_ret' , 'ret' , 'spy_ret', 'etf_ret' , 'volume']].corr()

## Obviously, predicting 'next_ret' is very challenging task
 # Remember the Efficient Market Hypothesis?


# The Efficient Market Hypothesis (EMH) implies that past information, including past returns, do not help predicting future return.

 # or the best price prediction of future price is the current price. 
  # This does not mean that then next price will be the same as current price. It means that unexpected events will cause such price changes.
  
  # Without the knowledge of these unexpected events, the current price contains all the information about the future price

## Nevertheless, day traders do try. 

 # Remember,

 # ( i )They have access to more information that we do, like book imbalances across markets;
 
  # ( i i ) While markets are efficient most of the time, short-lived pockets of inefficiencies can exist;
  
   # ( i i i ) better speed of execution, cheaper execution cost can make trading based such prediction a profitable effort.  successful trading requires 
   # (a) locating rare profitable opportunity, 
   # (b) monitoring inventory of stocks and,
   # (c) adjusting a stock trading trigers based on its prediction, its inventory, and its impact on the risk of the inventory portfolio. 

# Here we will focus on,
 # (i) using Linear regression
 # (ii) examine wht ANN can add to the prediction
 # (iii) Discuss whether clever feature engineering can help prediction without and with ANN

## 1. Simple regression model

In [ ]:
XY0= sym0[['ret', 'volume', 'next_ret']].dropna()
X0= XY0[['ret', 'volume']]
y = XY0['next_ret']
X0 = sm.add_constant(X0)
lm0 = sm.OLS(y, X0).fit()
lm0.summary()

In [ ]:
r = np.sqrt( lm0.rsquared )
round( r, 6)

## Not a great predictive model. Still we have a significant F stat

## the above regression assumes that ret and volume has additive effects.
 # What if there are non-linear effects
 # We call such effects interactions. For example, 
 # (i ) What if the slope of 'ret'with respect to 'next_ret' is different. For example, a strong move may cause reversion while mediocre move does not cause reversion
 # (ii ) What if strong volume and strong 'ret' has distinct impact on 'next_ret'.
 # ( iii ) what if a non-linear fumction of volume provide a predictive power that volume itself can't provide

## First let us examine the variables that capture the information on which we will base our prediction

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
syms0[['ret', 'volume', 'next_ret']].describe()

## Note that return and volume have different units. Some times standarizing the data can help improve prediction

# let us look at the distribution of 'ret'. Here we are importing seaborn and matplotlib and graphing some distributions

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
g= sns.displot(sym, x="ret" , kde=True)
g.fig.set_figwidth(6)
g.fig.set_figheight(3)
plt.xlim(0, 150000)
plt.xlim(-0.02, 0.02)

## Very much normal

## On the other hand the distribution of volume is very skewed

## First let us filter out zero volume data and then graph the distribution of volume

In [ ]:
syms= syms0[syms0['volume']> 0]
g = sns.displot(sym, x="volume" , kde=True)
g.fig.set_figwidth(6)
g.fig.set_figheight(3)
plt.xlim(syms['volume'].min(), syms['volume'].max())

## Here we limit the upper volume to its 95th quantile

In [ ]:

g = sns.displot(syms, x="volume" , kde=True)
g.fig.set_figwidth(6)
g.fig.set_figheight(3)
plt.xlim(syms['volume'].min(), syms['volume'].quantile(0.95))

## Very skewed distributution

## Standarization helps make all variables have the same scale.
 # some times we may want to transformsom variables so that their distributions are similar.
 
 # Let us introduce two transformations of volume by creating two columns:

# 1. This one creates a transformed variable witha zero mean and std=1. 

In [ ]:
syms['standaized_volume']= ( syms['volume'] - syms['volume'].mean() )/ syms['volume'].std()


## Let us see whether this distribution impacts skewness

In [ ]:
g= sns.displot(syms, x="standaized_volume" , kde=True)
g.fig.set_figwidth(6)
g.fig.set_figheight(5)
plt.xlim(0, 50)
plt.ylim(0, syms['standaized_volume'].max())

## THe above destribution changes location and scale unites. However, it does not take care of skewness

## Another transformation is taking natural log of volume. However, the log function is not defined for a zero argument. Let us filter out cases with zero volume

In [ ]:
syms['log_volume'] =  np.log(syms['volume'])

In [ ]:
g=sns.displot(syms, x="log_volume" , kde=True)
g.fig.set_figwidth(6)
g.fig.set_figheight(3)
plt.xlim(syms['log_volume'].min(), syms['log_volume'].max())

## Closer to being symmetric

## Some "manual" methods to capture non-linear effect:

 # I say manual because ANN is designed to capture non-linearity in the input variables

## Inntuitively, next_ret may response to strong up return differently that to mediocre return or strong negative return. Like the case of a pendulum, a weak force may have little impact, while a strong force results in an opposite direction swing. How can we test for that?

# We can engineer two features from the variable 'ret': (i) Capture the current return, 'ret' , when it is only strong up; and (ii) capture it when it is only strong down.

# we need threshold to determine up oand down. here is an example

## examine the following two variables


In [ ]:
q90 = syms['ret'].quantile(0.90)
q10 = syms['ret'].quantile(0.10)
syms['ret_q90'] = np.where(syms['ret'] > q90 , syms['ret']- q90 ,0 )
syms['ret_q10'] = np.where(syms['ret'] < q10 , syms['ret']- q10, 0 )
syms[['ret' , 'ret_q90' , 'ret_q10']].describe()

In [ ]:
syms[['next_ret','ret' , 'ret_q10' , 'ret_q90']].corr()

## Note how next_ret correlate differently between the three variables. Reversion is less pronounced for large current return. This is because sometime reversion happens and other time momentum happens with strong move. Some cancelation of reversion takes place

## Introducing Activation function ReLU ( Rectified Linear Unit). Recently most ML algorithms use such activation function

In [ ]:
def relu(x):
    y = np.maximum(0,x)
    return(y)

## Here is a graph of this function

In [ ]:
x= np.linspace(-0.01, 0.01, 100)
y = relu(x)
plt.figure(figsize=(6,4))
plt.plot(list(x),list(y) )
plt.title ( "y = relu(x)" ) 
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Here is a graph of the first engineered features

In [ ]:
syms.plot(kind= 'scatter' , x= 'ret', y='ret_q90', color='b', label = 'ret_q90 ')

## This is exactly relu(ret - q90 ). Let us verify that

In [ ]:
syms['relu_ret - q90'] = (syms['ret']-q90).apply(relu)
print( syms[['ret_q90', 'relu_ret - q90']].agg(['mean', 'std']) )
syms[['ret_q90', 'relu_ret - q90']].corr()

## This shows the two variables are identical. It also shows how the activstion function can be use to capture non linearity of any input any linear combinations of inputs

In [ ]:
syms.plot(kind= 'scatter' , x= 'ret', y='ret_q10', color='r', label = 'ret_q10 ')

In [ ]:

syms['r-act']= syms['ret'] -  (syms['ret']-q10).apply(relu)
syms.plot(kind= 'scatter' , x= 'ret', y='r-act', color='g', label = 'rrt - activation ')

## So relu activation function can also generate the second engineered feature

## By applying ReLU activiation function on a variable or a linear combination of variables we are engineering features that capture non-linear relationship between inputs and the out put variable

## What is ANN slow at or not efficient with is capturing interaction of a product effect between inputs, like ret*volume

# There are two ways to handle that;
 # (i) creating a new input equal to the product of the two inputs
 # ( ii) Apply a log transformation for the product. According to the log function,
 # log( a * b ) = log(a) + log(b)    if a > 0 and b > 0
 # If each term is positive, the log of the product is the sum of the logs. 
  # In case of the variable 'ret' , we face the problem that ret can be negative.
  
   #  However, if we can change ret to 1+ret. Using (1 +ret) instead of ret does not sacrifice any information. But it allows capturing the the interation, it neutralizes any impact of the skewed distribution of volume and helps ML algorithm learn about the effect of this interaction.
   # Another function of ret is exp(ret). If instead of ret*volume we have exp(ret)*volume, then 
   # log ( exp(ret)*volume ) = log(exp(ret) + log(volume) = ret + log(volume)

In [ ]:
syms['volume*ret']=syms['volume']*syms['ret']

In [ ]:
syms.columns

# Let us define the following df

In [ ]:
vrs= ['symbol', 'etf', 'day', 'time',  
       'next_ret', 'ret', 'spy_ret', 'etf_ret',
      'volume' , 'standaized_volume', 'log_volume','volume*ret',
      'ret_q90', 'ret_q10']
syms1= syms[vrs]
syms1.columns

In [ ]:
sym1_normalized = (syms1 - syms1.mean() )/syms1.std()
sym1_normalized.describe()

## Here is a regression model with all the feature

In [ ]:
print(vrs)

In [ ]:
print( vrs[4:] )

In [ ]:
XY_syms1 = syms1[vrs[4:]].dropna()
X= XY_syms1[vrs[5:]]
y = XY_syms1['next_ret']
X = sm.add_constant(X)
lm1 = sm.OLS(y, X).fit()
lm1.summary()

In [ ]:
np.sqrt(lm1.rsquared)

## some improvement in r. Lower F-stat. Interaction between volume and return is significant. volume itself ( including the two transformed volumes) is not significant

## Out-of-sample testing
 # Split the data to train and test
 # standarizing train-data
 # use train parameters to standarize test-data

In [ ]:
syms1.columns

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [ ]:
y = syms1['next_ret']

In [ ]:
syms1_train, syms1_test, y_train, y_test = train_test_split(syms1, y, test_size=0.3, random_state= 12345)

In [ ]:
vrs= syms1_train.columns
vrs

In [ ]:
XY_train = syms1_train[vrs[4:]].dropna()
XY_test = syms1_test[vrs[4:]].dropna()
XY_train_standarized = (XY_train - XY_train.mean() )/XY_train.std() 
XY_test_standarized = (XY_test - XY_train.mean() )/XY_train.std()  # Note we standarize the test data with the 
X_train_standarized= XY_train_standarized[vrs[5:]]
X_test_standarized = XY_test_standarized[vrs[5:]]
y_train_standarized = XY_train_standarized['next_ret']
y_test_standarized =   XY_test_standarized['next_ret']

X_train_standarized = sm.add_constant(X_train_standarized)
X_test_standarized = sm.add_constant(X_test_standarized)
lm = sm.OLS(y_train_standarized, X_train_standarized).fit()
XY_train_standarized['pred'] = lm.predict(X_train_standarized)
XY_test_standarized['pred'] = lm.predict(X_test_standarized)
r_train = XY_train_standarized['pred'].corr(XY_train_standarized['next_ret'])
r_test = XY_test_standarized['pred'].corr(XY_test_standarized['next_ret'])

print(  'r_train = = ' , r_train)
print(  'r_test    =' , r_test )

In [ ]:
print('train_rsquared = ' ,f'{r_train**2:.8f}'  )
print('test_rsquared = ' , f'{r_test**2:.8f}' )

## It is lower than the r-square of the regression without splitting the sample. Hoowever, it is very close, due to the large sample size

## 2. Introduction to Keras

## We should expect ANN to take care of non-linearity. 
## we will apply sequential ANN using raw basic variables without interactionsl
  # Specifically [ 'ret' , spy_ret , etf_ret , volume ]
  ## The point here see whether ANN can do all the work for us
## We will run ANN with all created interactions and see how much ANN improves ove our model with our manually developed interactions

 ## First, we 
#  (i) will install and use two python library : tensorflow and keras
 # Provide a description of Keras sequantial NN parameters 

In [ ]:
!pip install tensorflow
!pip install keras

## A tensor is a multi-dimensional Array (including 2 or more dimesions)
## TensorFlow is an open source library for numerical computation and large-scale machine learning. 
 # It bundles Neural Network algorithms. 
 # It uses python to provide a convenient front-end API for building models. However, those applications get executed  C++

## TensorFlow allows creating data flow graphs. That is:
 # A structures that describe how data moves through a graph. 
 # Each node in the graph represents a mathematical operation.
 # Each connection or edge between nodes is a multi-dimensional data array-- tensor.

## Import these keras libraries

In [ ]:

from keras.models import Sequential
from keras.layers import Dense

## With the “Sequential” we create a sequence of ANN layers stacked one after the other. 
# Each layer is defined using the “Dense” module of Keras.
# Keras Dense layer contains all the neurons that are deeply connected within themselves. Every neuron in the dense layer takes the input from all the other neurons of the previous layer.  
# We need to specify,
 # ( i) how many neurons would be there, 
 # (Ii) How we initialize the weights in the network. 
 # (iii) the activation function for each neuron in that layer

## Parameters to be specified:
  # 1. units=8 means we are creating a layer with 5 neirons. Each of of which will be receiving input values

## input_dim=4: 
 # This means there are 4 predictors in the input data which is expected by the first layer. 
 # The Sequential model passes this information further to the next layers.


## kernel_initializer=’normal’: 
 # The algorithm has to decide the value for each weight. This parameter specifies that. You can choose different values for instead of ‘normal’ . The following link provides all the options:
 https://keras.io/api/layers/initializers/


## activation=’relu’: This specifies the activation function for the calculations inside each neuron. You can choose values like ‘relu’, ‘tanh’, ‘sigmoid’, etc.

## batch_size=60: 
 # This specifies how many rows will be passed to the Network in one go after which the SSE calculation will begin and the neural network will start adjusting its weights based on the errors.
 # When all the rows are passed in the batches of 60 rows each as specified in this parameter, then we call that 1-epoch. Or one full data cycle. 
 # This is also known as mini-batch gradient descent. 
 # A small value of batch_size will make the ANN look at the data slowly, 
 # a large value will make the ANN look at the data fast which could lead to underfitting.
 # a proper value must be chosen using hyperparameter tuning.

## Epochs=50: 
## The same activity of adjusting weights continues for 50 times, as specified by this parameter. 
# That isthe ANN looks at the full training data 50 times and adjusts its weights.

In [ ]:
print(vrs)

In [ ]:
vrs[5:9]

In [ ]:
ann_basic_columns = [ 'ret' , 'volume' , 'spy_ret' , 'etf_ret' ]
x_train = X_train_standarized[ann_basic_columns]
x_test = X_test_standarized[ann_basic_columns]
y_train = XY_train_standarized['next_ret']
y_test = XY_test_standarized['next_ret']
print (x_train.shape, x_test.shape , y_train.shape , y_test.shape )

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
 
# create ANN model
model = Sequential()
 
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=12, input_dim= 4, kernel_initializer='normal', activation='relu'))
 
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
 
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
 
# Compiling the model
from sklearn.metrics import r2_score
model.compile(loss='mse', optimizer='rmsprop', metrics=['cosine_proximity'], run_eagerly=True)
# model.compile(loss='mean_squared_error', optimizer='adam')
 
# Fitting the ANN to the Training set
model.fit(x_train, y_train ,batch_size = 60, epochs = 50, verbose=1)

## cosine proximity is corelation centered arround zero. Actually correlation between x and y is cosine proximity centered arount (x_mean, y_mean)

## Here we create a prediction column in the standarized-train-data and calculate the in-sample correlation coefficient between next_re and its prediction

In [ ]:
XY_train_standarized ['basic_train_pred'] =  model.predict(x_train) 
XY_train_standarized ['basic_train_pred'].corr( XY_train_standarized ['next_ret'] )  

## here we do the same for the out-of-sample correlation between next_ret and its prediction

In [ ]:
XY_test_standarized ['basic_test_pred'] =  model.predict(x_test) 
XY_test_standarized ['basic_test_pred'].corr( XY_test_standarized ['next_ret'] ) 

## it lokas like ANN delivers better out-of-sample results without having to include the manual interactions in the inputs. ANN took care of the interactions itself

## What if we specify interaction variables in the input ? 
 # Can we further enhance the performance?

In [ ]:
print( vrs[5:] )

In [ ]:
X_train_standarized.columns

In [ ]:
ann_columns = vrs[5:]
x_train = X_train_standarized[ann_columns]
x_test = X_test_standarized[ann_columns]
y_train = XY_train_standarized['next_ret']
y_test = XY_test_standarized['next_ret']
print (x_train.shape, x_test.shape , y_train.shape , y_test.shape )

In [ ]:
model = Sequential()

model.add(Dense(units=12, input_dim= 9, kernel_initializer='normal', activation='relu'))

model.add(Dense(units=4, kernel_initializer='normal', activation='relu'))

model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mse', optimizer='rmsprop', metrics=['cosine_proximity'], run_eagerly=True)
model.fit(x_train, y_train ,batch_size = 60, epochs = 50, verbose=1)

In [ ]:
XY_train_standarized ['ann_train_pred'] =  model.predict(x_train) 
XY_train_standarized ['ann_train_pred'].corr( XY_train_standarized ['next_ret'] )  

In [ ]:
XY_test_standarized ['ann_test_pred'] =  model.predict(x_test) 
XY_test_standarized ['ann_test_pred'].corr( XY_test_standarized ['next_ret'] )  

## Two conclusions
 # (i) the basic ( with 4 basic inputs)  Sequential ANN model does better than a linear regression that includes the interaction terms ( 9 inputs in total)
 # (ii) Adding the interaction variables to the inputs of the Sequential ANN did enhance out-of-sample performance. 
 
 ## This testifies to the power of ANN

In [ ]:
syms.columns

In [ ]:
vrs_all = ['symbol', 'etf', 'day', 'time','next_ret', 'ret',
       'prev1_ret', 'prev2_ret', 'prev3_ret', 'prev4_ret', 'volume', 'mflow1',
       'mflow2', 'mflow3', 'mflow4', 'spy_high', 'spy_low', 
       'spy_ret',  'spy_prev3_ret',
       'spy_prev4_ret', 'spy_volume', 'spy_mflow1', 'spy_mflow2', 'spy_mflow3',
       'spy_mflow4', 'etf_high', 'etf_low',  'etf_ret',
       'etf_prev1_ret', 'etf_prev2_ret', 'etf_prev3_ret', 'etf_prev4_ret',
       'etf_volume', 'etf_mflow1', 'etf_mflow2', 'etf_mflow3', 'etf_mflow4',
       ]

In [ ]:
syms1_train.columns

In [ ]:
syms.shape

In [ ]:
XY = syms[vrs_all[4:]].dropna()
X= XY[vrs_all[5:]]
y = XY['next_ret']
X = sm.add_constant(X)
lm1 = sm.OLS(y, X).fit()
lm1.summary()

In [ ]:
y = syms['next_ret']
syms_train, syms_test, y_train, y_test = train_test_split(syms, y, test_size=0.3, random_state= 12345)
XY_train = syms_train[vrs_all[4:]].dropna()
XY_test = syms_test[vrs_all[4:]].dropna()

In [ ]:
XY_train = syms_train[vrs_all[4:]].dropna()
XY_test = syms_test[vrs_all[4:]].dropna()
XY_train_standarized = (XY_train - XY_train.mean() )/XY_train.std() 
XY_test_standarized = (XY_test - XY_train.mean() )/XY_train.std()  # Note we standarize the test data with the 
X_train_standarized= XY_train_standarized[vrs_all[5:]]
X_test_standarized = XY_test_standarized[vrs_all[5:]]
y_train_standarized = XY_train_standarized['next_ret']
y_test_standarized =   XY_test_standarized['next_ret']

y_train = XY_train_standarized['next_ret']
y_test = XY_test_standarized['next_ret']

X_train_standarized = sm.add_constant(X_train_standarized)
X_test_standarized = sm.add_constant(X_test_standarized)
lm = sm.OLS(y_train_standarized, X_train_standarized).fit()
XY_train_standarized['pred'] = lm.predict(X_train_standarized)
XY_test_standarized['pred'] = lm.predict(X_test_standarized)
r_train = XY_train_standarized['pred'].corr(XY_train_standarized['next_ret'])
r_test = XY_test_standarized['pred'].corr(XY_test_standarized['next_ret'])
print(XY_train_standarized.shape , XY_test_standarized.shape)
print(  'r_train = = ' , r_train)
print(  'r_test    =' , r_test )


## This provides imprtovement over the earlier 9-independent variables regression model that used onlu current information. The variables here information go back four 5-minutes periods.  # It captures past returns and money flow.
# money-flow is defined here as return times volume. It is actually interaction of volume and returns for the symbol, its etf and spy
# A more accurate definition for money flow is price change times volume. Here it is volume times percentage price changes (i.e. return)

In [ ]:
print(X_train_standarized.columns)
print ( X_train_standarized.shape )

In [ ]:
print(XY_train_standarized.columns )

In [ ]:
print(XY_test_standarized.columns )

## Before we start, we can transform the standarized next_ret and its prediction back to their original values.
 # Recall that we divided subtracted the XY_train['next_ret'].mean() and divided by XY_train['next_ret'].std() . The reverse transformation would  multiplying by std and adding the mean. The followin renames XY_?? data frames and calculate original values for next_ret and apply the same transformation to the prediction

In [ ]:

LM32_XY_train = XY_train_standarized
LM32_XY_test = XY_test_standarized
next_ret_train_mean = XY_train['next_ret'].mean()
next_ret_train_std = XY_train['next_ret'].std()
LM32_XY_train['original_next_ret'] = next_ret_train_std *LM32_XY_train['next_ret']+ next_ret_train_mean 
LM32_XY_train['original_scale_pred'] = next_ret_train_std *LM32_XY_train['pred']+ next_ret_train_mean 

LM32_XY_test['original_next_ret'] = next_ret_train_std *LM32_XY_test['next_ret']+ next_ret_train_mean 
LM32_XY_test['original_scale_pred'] = next_ret_train_std *LM32_XY_test['pred']+ next_ret_train_mean 

In [ ]:
LM32_XY_train[['next_ret',  'pred', 'original_next_ret', 'original_scale_pred'] ].describe()

In [ ]:
LM32_XY_test['pred_Quantile'] = pd.qcut(LM32_XY_test.pred,5 ,        
        labels=['Q1',
                'Q2',
                'Q3',
                'Q4', 'Q5'])

In [ ]:
LM32_next_ret_byQ = LM32_XY_test.groupby('pred_Quantile')['original_next_ret'].agg(['mean', 'std', 'count'])
LM32_next_ret_byQ['t_stat'] = LM32_next_ret_byQ['mean']/(LM32_next_ret_byQ['std']/LM32_next_ret_byQ['count']**0.5)
LM32_next_ret_byQ

## Note that the 'pred' std is much smaller than next_ret-std. This carries on after the transformation
 # The scale of the prediction is lower as long as the regressions r-square is less than 1. The samler r_square the larger the scale disparity.

## Comments on trading

## If you short the first and second quatimes and long the 4th and the fifth quatile you can make money
 # There are 78 5-minutes window in a day. we can actually calculate the daily return and approximate daily std

## Assume thatwe we short $1 of quatile 1 and 2 and long $1 in quantile 4 and 45 the average 5-minutes return is

In [ ]:
annual_avg_ret = ( ( -LM32_next_ret_byQ['mean'][0] - LM32_next_ret_byQ['mean'][1] +
   LM32_next_ret_byQ['mean'][3] + LM32_next_ret_byQ['mean'][4] )/4 )*78*252
annual_std_ret =np.sqrt (   ( LM32_next_ret_byQ['std'][0]**2 + LM32_next_ret_byQ['std'][1]**2 +
   LM32_next_ret_byQ['std'][3]**2 + LM32_next_ret_byQ['std'][4]**2 )/4  )*np.sqrt(78*252)
print( 'annual_avg_ret = ',  annual_avg_ret  )
print( 'annual_std_ret = ',  annual_std_ret  )  
print( 'sharpe-ratio = ' ,  annual_avg_ret/annual_std_ret )

## This can be optimistic, it ignores transaction cost and assumes that you can trade at the begining of the 5-minutes windo at the open price and set at the close price that window.
 # a more accurate analysis require simulation testing and an inventory management strategy.  Such staretegy may decide to seel or keep the position

## Can ANN do a better job if we feed it the same inputs?

## The following code replaces XY_?? data frames. However, their information is in LM32_??

In [ ]:
print(vrs_all)

In [ ]:
XY_train = syms_train[vrs_all[4:]].dropna()
XY_test = syms_test[vrs_all[4:]].dropna()

In [ ]:
XY_train = syms_train[vrs_all[4:]].dropna()
XY_test = syms_test[vrs_all[4:]].dropna()
ann_columns = vrs_all[5:]
x_train = X_train_standarized[ann_columns]
x_test = X_test_standarized[ann_columns]
y_train = XY_train_standarized['next_ret']
y_test = XY_test_standarized['next_ret']
print (x_train.shape, x_test.shape , y_train.shape , y_test.shape )

In [ ]:

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()

model.add(Dense(units= 40, input_dim= 32, kernel_initializer='normal', activation='relu'))

model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mse', optimizer='rmsprop', metrics=['cosine_proximity'], run_eagerly=True)
model.fit(x_train, y_train ,batch_size = 200, epochs = 10, verbose=1)

In [ ]:
XY_train_standarized ['ann_train_pred'] =  model.predict(x_train) 
XY_train_standarized ['ann_train_pred'].corr( XY_train_standarized ['next_ret'] )  

In [ ]:
XY_test_standarized ['ann_test_pred'] =  model.predict(x_test) 
XY_test_standarized ['ann_test_pred'].corr( XY_test_standarized ['next_ret'] ) 

## Very impressive performance. The out of sample correlation between actual and prediction   is almost three times its linear model with ellaborite manual interactions 

## Note howver, that without developing these interactions, the ANN performance is rather limited

## The conclusion here is that you need to study and understand the problem, engineer features from such understanding and then apply ANN

## This ANN can be optimized with hyper-parameter calibration and more data

## Impact on trading

In [ ]:
XY_test_standarized.columns

In [ ]:
ANN32_XY_train = XY_train_standarized
ANN32_XY_test = XY_test_standarized
next_ret_train_mean = XY_train['next_ret'].mean()
next_ret_train_std = XY_train['next_ret'].std()
ANN32_XY_train['original_next_ret'] = next_ret_train_std *ANN32_XY_train['next_ret']+ next_ret_train_mean 
ANN32_XY_train['original_scale_pred'] = next_ret_train_std *ANN32_XY_train['pred']+ next_ret_train_mean 

ANN32_XY_test['original_next_ret'] = next_ret_train_std *ANN32_XY_test['next_ret']+ next_ret_train_mean 
ANN32_XY_test['original_scale_pred'] = next_ret_train_std *ANN32_XY_test['pred']+ next_ret_train_mean 

In [ ]:
ANN32_next_ret_byQ = ANN32_XY_test.groupby('pred_Quantile')['original_next_ret'].agg(['mean', 'std', 'count'])
ANN32_next_ret_byQ['t_stat'] = ANN32_next_ret_byQ['mean']/(ANN32_next_ret_byQ['std']/np.sqrt(ANN32_next_ret_byQ['count']) )
ANN32_next_ret_byQ

In [ ]:
ANN32_XY_test['pred_Quantile'] = pd.qcut(ANN32_XY_test.ann_test_pred,5 ,        
        labels=['Q1',
                'Q2',
                'Q3',
                'Q4', 'Q5'])
annual_avg_ret = ( ( -ANN32_next_ret_byQ['mean'][0] - ANN32_next_ret_byQ['mean'][1] +
   ANN32_next_ret_byQ['mean'][3] + ANN32_next_ret_byQ['mean'][4] )/4 )*78*252
annual_std_ret =np.sqrt (   ( ANN32_next_ret_byQ['std'][0]**2 + ANN32_next_ret_byQ['std'][1]**2 +
   ANN32_next_ret_byQ['std'][3]**2 + ANN32_next_ret_byQ['std'][4]**2 )/4  )*np.sqrt(78*252)
print( 'annual_avg_ret = ',  annual_avg_ret  )
print( 'annual_std_ret = ',  annual_std_ret  )  
print( 'sharpe-ratio = ' ,  annual_avg_ret/annual_std_ret )

## The ANN does provide much better psueudo trading results. 219% instead of 80% return with similar risk. It triples the sharpe ration